<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Collaborative_Filtering_Hotels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
import pandas as pd

# Load user preferences data
user_preferences_df  = pd.read_csv('/content/drive/MyDrive/Filtering_Data/user_preferences.csv', encoding='utf-8')

# Load restaurant data
hotels_df = pd.read_csv('/content/drive/MyDrive/Filtering_Data/content_filtered_data_hotel.csv', encoding='utf-8')

In [65]:
hotels_df.head(5)

,Unnamed: 0.1,Unnamed: 0,property_name,address,area,city,hotel_description,hotel_facilities,hotel_star_rating,image_urls,...,longitude,pageurl,property_type,province,room_count,room_facilities,room_type,state,rating,Distance
0,15930,15930,Shilton Suites,"No:2, Ulsoor Road (Parallel to MG Road) , Ulso...","Ulsoor,Central Bangalore",Bangalore,"|Situated in the Silicon Valley of India, Shil...",Airport/Rlwy Stn Transfer|Business Centre|Curr...,2.0,//imghtlak.mmtcdn.com/hotels/20110121180320201...,...,77.617424,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe Suite,Telangana,4.5,0.047548
1,12785,12785,Nandhana Comforts Marathahalli,"No 1, 1st Floor, Outer Ring Rd, Near to Relien...",Marathahalli,Bangalore,"Nestled in the city of Bangalore, Hotel Nandha...",Doctor on Call|Laundry Service|Restaurant|Room...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014102017...,...,77.713085,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Standard|Deluxe,Karnataka,4.5,0.053065
2,17759,17759,The Libra Hotel,"11/1, Hosur Main Road, In-Between Christ Unive...",Koramangala,Bangalore,Around 4 km from Lal Bagh and 9 km from the ra...,Airport Transfer|Doctor on Call|Laundry Servic...,2.0,//imghtlak.mmtcdn.com/images/hotels/2015090711...,...,77.609272,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Economy Ac|Executive Ac|Premium Ac,Karnataka,4.5,0.065624
3,1378,1378,Boulevard Residence,"Thanisandra Main Road,ICICI Thanisandra Main road",Hebbal,Bangalore,Boulevard Residence is nestled in Bangalore po...,24 hour security|24 hours power back-up|Airpor...,2.0,//imghtlak.mmtcdn.com/images/hotels/2011120617...,...,77.624625,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Standard Room,Karnataka,4.5,0.095512
4,17599,17599,The Grand Continent,"44B, 1st Main Road, Opp. Mini Forest,3rd Phase...",JP Nagar,Bangalore,Situated at a 20 minute drive from Bangalore C...,Airport/Rlwy Stn Transfer|Coffee Shop|Conferen...,2.0,//imghtlak.mmtcdn.com/images/hotels/2007012519...,...,77.599166,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe King,Karnataka,4.5,0.087935


In [66]:
# Rename the column "hotel_facilities" to "facilities"
hotels_df = hotels_df.rename(columns={'hotel_facilities': 'facilities'})

In [67]:
hotels_df = hotels_df.rename(columns={'property_name': 'Hotel_name'})

In [68]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer


# Define a function to calculate Jaccard similarity between a user's preferences and all hotels
def calculate_similarity(user_id):
    user_prefs = user_preferences_df[user_preferences_df['user_id'] == user_id].iloc[:, 3:].values
    hotel_features = hotels_df.iloc[:, 3:].values

    # Create a MultiLabelBinarizer object to one-hot encode the values in each column as binary features
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on all the values in the 'Facilities' column of the hotels dataframe
    mlb.fit(hotels_df['facilities'].str.split(', '))

    # Transform the 'Facilities' column of both user preferences and hotels dataframes into binary features
    user_facilities = mlb.transform(user_preferences_df[user_preferences_df['user_id'] == user_id]['facilities'].str.split(', '))
    hotel_facilities = mlb.transform(hotels_df['facilities'].str.split('|'))

    # Calculate Jaccard similarity between the user's preferences and all hotels
    similarities = pairwise_distances(user_facilities, hotel_facilities, metric='jaccard')

    return 1 - similarities[0]


In [69]:
hotels_df

,Unnamed: 0.1,Unnamed: 0,Hotel_name,address,area,city,hotel_description,facilities,hotel_star_rating,image_urls,...,longitude,pageurl,property_type,province,room_count,room_facilities,room_type,state,rating,Distance
0,15930,15930,Shilton Suites,"No:2, Ulsoor Road (Parallel to MG Road) , Ulso...","Ulsoor,Central Bangalore",Bangalore,"|Situated in the Silicon Valley of India, Shil...",Airport/Rlwy Stn Transfer|Business Centre|Curr...,2.0,//imghtlak.mmtcdn.com/hotels/20110121180320201...,...,77.617424,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe Suite,Telangana,4.5,0.047548
1,12785,12785,Nandhana Comforts Marathahalli,"No 1, 1st Floor, Outer Ring Rd, Near to Relien...",Marathahalli,Bangalore,"Nestled in the city of Bangalore, Hotel Nandha...",Doctor on Call|Laundry Service|Restaurant|Room...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014102017...,...,77.713085,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Standard|Deluxe,Karnataka,4.5,0.053065
2,17759,17759,The Libra Hotel,"11/1, Hosur Main Road, In-Between Christ Unive...",Koramangala,Bangalore,Around 4 km from Lal Bagh and 9 km from the ra...,Airport Transfer|Doctor on Call|Laundry Servic...,2.0,//imghtlak.mmtcdn.com/images/hotels/2015090711...,...,77.609272,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Economy Ac|Executive Ac|Premium Ac,Karnataka,4.5,0.065624
3,1378,1378,Boulevard Residence,"Thanisandra Main Road,ICICI Thanisandra Main road",Hebbal,Bangalore,Boulevard Residence is nestled in Bangalore po...,24 hour security|24 hours power back-up|Airpor...,2.0,//imghtlak.mmtcdn.com/images/hotels/2011120617...,...,77.624625,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Standard Room,Karnataka,4.5,0.095512
4,17599,17599,The Grand Continent,"44B, 1st Main Road, Opp. Mini Forest,3rd Phase...",JP Nagar,Bangalore,Situated at a 20 minute drive from Bangalore C...,Airport/Rlwy Stn Transfer|Coffee Shop|Conferen...,2.0,//imghtlak.mmtcdn.com/images/hotels/2007012519...,...,77.599166,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe King,Karnataka,4.5,0.087935
5,15181,15181,S K Elegance,"2722/1, L13/1 L/14, Sri Harsha Road , Mysore",Lasker Mohalla,Mysore,"|Ideally located on Sri Harsha Road, SK Elegan...",Airport Transfer|Doctor on Call|Guide Service|...,2.0,//imghtlak.mmtcdn.com/images/hotels/2012091018...,...,76.658773,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Executive Non A/c|Executive Ac,Karnataka,4.5,1.204101
6,17110,17110,TGI Star Holidays,"Opp Union Office, Salem Main , Road Yercaud",Yercaud,Yercaud,|TGI Star Holidays is a budget property locate...,24 hour room service|24 hour security|24 hours...,2.0,//imghtlak.mmtcdn.com/images/hotels/2010031216...,...,78.203827,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Luxury|Courtyard/pool View Room|Lake Room|Tril...,Kerala,4.5,1.303057
7,7733,7733,Hotel Naveen Residency,"Tower Chowk ,Near Naveen Vastralaya, Darbhanga",Dharbhanga,Dharbhanga,HOTEL RADHA PRASAD ERODE has 67 well furnished...,Food &amp; Beverage:Restaurant|Travel:Travel D...,2.0,http://ui.cltpstatic.com/places/hotels/1331/13...,...,77.722353,http://www.cleartrip.com/hotels/details/133168...,Hotel,Dharbhanga,38.0,Television | Telephone | In-Room Menu | Privat...,Standard Non AC Room,Bihar,4.5,1.631845
8,19912,19912,West Land Hotel,"# 399,B. Perundurai Road, Opp Senthil Multispe...",Near Bus Terminus,Erode,"West Inn Hotel, Erode is an exotic hotel with ...",Basics:Internet|Air Conditioning|Lift|Express ...,2.0,http://ui.cltpstatic.com/places/hotels/7150/71...,...,77.714401,http://www.cleartrip.com/hotels/details/715020...,Hotel,Erode,36.0,Air Conditioning| Television| Iron| Telephone ...,Executive Room,TamilNadu,4.5,1.632553
9,4598,4598,HOTEL SLV GRAND,Beside Ttd Srinivasam COMPLEX Tirumala Bye Pas...,Bye Pass Road Tirupathi,WeekendGetawaysAroundChennai,"|Nestled in the beautiful heritage city, Tirup...",24 hour r

In [70]:
# Remove unwanted columns
hotels_df = hotels_df.drop(['Unnamed: 0'], axis=1)


In [71]:
hotels_df = hotels_df.drop(['Unnamed: 0.1'], axis=1)
hotels_df

,Hotel_name,address,area,city,hotel_description,facilities,hotel_star_rating,image_urls,locality,latitude,longitude,pageurl,property_type,province,room_count,room_facilities,room_type,state,rating,Distance
0,Shilton Suites,"No:2, Ulsoor Road (Parallel to MG Road) , Ulso...","Ulsoor,Central Bangalore",Bangalore,"|Situated in the Silicon Valley of India, Shil...",Airport/Rlwy Stn Transfer|Business Centre|Curr...,2.0,//imghtlak.mmtcdn.com/hotels/20110121180320201...,NaN,12.975349,77.617424,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe Suite,Telangana,4.5,0.047548
1,Nandhana Comforts Marathahalli,"No 1, 1st Floor, Outer Ring Rd, Near to Relien...",Marathahalli,Bangalore,"Nestled in the city of Bangalore, Hotel Nandha...",Doctor on Call|Laundry Service|Restaurant|Room...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014102017...,NaN,12.948576,77.713085,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Standard|Deluxe,Karnataka,4.5,0.053065
2,The Libra Hotel,"11/1, Hosur Main Road, In-Between Christ Unive...",Koramangala,Bangalore,Around 4 km from Lal Bagh and 9 km from the ra...,Airport Transfer|Doctor on Call|Laundry Servic...,2.0,//imghtlak.mmtcdn.com/images/hotels/2015090711...,NaN,12.935282,77.609272,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Economy Ac|Executive Ac|Premium Ac,Karnataka,4.5,0.065624
3,Boulevard Residence,"Thanisandra Main Road,ICICI Thanisandra Main road",Hebbal,Bangalore,Boulevard Residence is nestled in Bangalore po...,24 hour security|24 hours power back-up|Airpor...,2.0,//imghtlak.mmtcdn.com/images/hotels/2011120617...,NaN,13.057086,77.624625,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Standard Room,Karnataka,4.5,0.095512
4,The Grand Continent,"44B, 1st Main Road, Opp. Mini Forest,3rd Phase...",JP Nagar,Bangalore,Situated at a 20 minute drive from Bangalore C...,Airport/Rlwy Stn Transfer|Coffee Shop|Conferen...,2.0,//imghtlak.mmtcdn.com/images/hotels/2007012519...,NaN,12.911775,77.599166,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe King,Karnataka,4.5,0.087935
5,S K Elegance,"2722/1, L13/1 L/14, Sri Harsha Road , Mysore",Lasker Mohalla,Mysore,"|Ideally located on Sri Harsha Road, SK Elegan...",Airport Transfer|Doctor on Call|Guide Service|...,2.0,//imghtlak.mmtcdn.com/images/hotels/2012091018...,NaN,12.308620,76.658773,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Executive Non A/c|Executive Ac,Karnataka,4.5,1.204101
6,TGI Star Holidays,"Opp Union Office, Salem Main , Road Yercaud",Yercaud,Yercaud,|TGI Star Holidays is a budget property locate...,24 hour room service|24 hour security|24 hours...,2.0,//imghtlak.mmtcdn.com/images/hotels/2010031216...,NaN,11.784091,78.203827,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Luxury|Courtyard/pool View Room|Lake Room|Tril...,Kerala,4.5,1.303057
7,Hotel Naveen Residency,"Tower Chowk ,Near Naveen Vastralaya, Darbhanga",Dharbhanga,Dharbhanga,HOTEL RADHA PRASAD ERODE has 67 well furnished...,Food &amp; Beverage:Restaurant|Travel:Travel D...,2.0,http://ui.cltpstatic.com/places/hotels/1331/13...,"Dharbhanga, Dharbhanga",11.339567,77.722353,http://www.cleartrip.com/hotels/details/133168...,Hotel,Dharbhanga,38.0,Television | Telephone | In-Room Menu | Privat...,Standard Non AC Room,Bihar,4.5,1.631845
8,West Land Hotel,"# 399,B. Perundurai Road, Opp Senthil Multispe...",Near Bus Terminus,Erode,"West Inn Hotel, Erode is an exotic hotel with ...",Basics:Internet|Air Conditioning|Lift|Express ...,2.0,http://ui.cltpstatic.com/places/hotels/7150/71...,"Near Bus Terminus, Erode",11.338597,77.714401,http://www.cleartrip.com/hotels/details/715020...,Hotel,Erode,36.0,Air Conditioning| Television| Iron| Telephone ...,Executive Room,TamilNadu,4.5,1.632553
9,HOTEL SLV GRAND,Beside Ttd Srinivasam COMPLEX Tirumala Bye Pas...,Bye Pass Road Tirupathi,WeekendGetawaysAroundChennai,"|Nestled in the beautiful heritage city, Tirup..."

In [ ]:
hotel_names = hotels_df['Hotel_name']

In [73]:
print(hotels_df[hotels_df.duplicated(subset=['Hotel_name'], keep=False)])


Empty DataFrame
Columns: [Hotel_name, address, area, city, hotel_description, facilities, hotel_star_rating, image_urls, locality, latitude, longitude, pageurl, property_type, province, room_count, room_facilities, room_type, state, rating, Distance]
Index: []


In [84]:
# Example usage: Recommend top 5 hotels for user 1
user_id = 1
similarities = calculate_similarity(user_id)
hotels_data = hotels_df[['Hotel_name', 'facilities','property_type','rating', 'Distance']]
recommendations = pd.DataFrame({'Hotel_name': hotel_names, 'Similarity': similarities})


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['24-Hour Room Service', 'Air Conditioning', 'Airport Shuttle', 'Bar', 'Business Center', 'Free Parking', 'Gym', 'Housekeeping', 'Laundry Service', 'Pet-Friendly', 'Swimming Pool'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', ' 24-hr Room Service ', ' Air Conditioned ', ' Air Conditioning', ' Air Conditioning ', ' Air conditioning ', ' Attached Bathroom', ' Attached Bathroom ', ' Balcony ', ' Bath Tub ', ' Bathrobe ', ' Bathrobes', ' Bathrobes ', ' Bathroom Toiletries ', ' Bathtub ', ' Broadband Internet ', ' Cable T V', ' Cable Tv', ' DVD Player ', ' DVD Player on Request ', ' Daily Newspaper ', ' Desk ', ' Direct dial phone ', ' Direct-dial phone ', ' Electronic Safe ', ' Equipped Kitchenette ', ' Fireplace ', ' Full-length mirror ', ' Geyser In Bathroom ', ' Hair Dryer ', ' Hot &

In [85]:
recommendations = recommendations.merge(hotels_data, on='Hotel_name').sort_values(by='Similarity', ascending=False).head(5)
print(recommendations)

                    Hotel_name  Similarity  \
0               Shilton Suites         1.0   
24   Woodway Estate - Homestay         1.0   
26       Hiy Rooms in GCT Road         1.0   
27       Hiy Rooms in RS Puram         1.0   
28  Hotel Balaji International         1.0   

                                           facilities property_type  rating  \
0   Airport/Rlwy Stn Transfer|Business Centre|Curr...         Hotel     4.5   
24  Travel:Travel Desk|Transportation Service|Pers...      Homestay     4.5   
26  Basics:Internet|Air Conditioning|Express Check...        Resort     4.5   
27  Basics:Internet|Air Conditioning|Express Check...      Homestay     4.5   
28  Airport Transfer|Bar|Bicycles on Hire|Board Ro...         Hotel     4.5   

    Distance  
0   0.047548  
24  2.024875  
26  2.070393  
27  2.057287  
28  2.070634  


In [87]:
print(recommendations.to_string(index=False))

                Hotel_name  Similarity                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  